In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import pandas
data=pd.read_csv("/kaggle/input/bri-data-hackathon-pa/train.csv")
datatest=pd.read_csv("/kaggle/input/bri-data-hackathon-pa/test.csv")
data=data.dropna()
print(datatest.head())
print(data.isna().sum())
print(data.columns.tolist())
print(datatest.index)

In [ ]:
print(data.dtypes)
data.describe()

In [ ]:
datadrop=data['GPA'][(data['GPA']>0)&(data['GPA']<4)]
#data['GPA'][data['GPA']==0]=datadrop.median()
#data['GPA'][data['GPA']>4]=datadrop.median()
data.loc[data['GPA']==0,'GPA']=datadrop.median()
data.loc[data['GPA']>4,'GPA']=datadrop.median()
print(data['GPA'])
print(datadrop.median())
data['GPA'].describe()

In [ ]:
datadroptes=datadrop.append(datatest['GPA'][(datatest['GPA']>0)&(datatest['GPA']<4)])

#data['GPA'][data['GPA']==0]=datadrop.median()
#data['GPA'][data['GPA']>4]=datadrop.median()
datatest.loc[datatest['GPA']==0,'GPA']=datadroptes.median()
datatest.loc[datatest['GPA']>4,'GPA']=datadroptes.median()
print(datatest['GPA'])
print(datadroptes)
print(datadroptes.median())
datatest['GPA'].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(data['GPA'],hist=False)
sns.distplot(data['GPA'][data['Best Performance']==1],hist=False,color='g')
plt.title("gpa")
plt.show()

In [ ]:
datates=data.dropna(axis=0)
#dropin data yang dtypenya object (string)
datates=datates.drop(['job_level','person_level','Employee_type','marital_status_maried(Y/N)','Education_level'],axis=1)
Xnya=datates.drop('Best Performance',axis=1)
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
kbes=SelectKBest()
hasil=kbes.fit(Xnya,datates['Best Performance'])
result=pd.concat([pd.DataFrame(Xnya.columns),pd.DataFrame(hasil.scores_)],axis=1)
result.columns=['Parameter','Score']
print('with f classif (ANOVA)')
print(result.sort_values('Score',ascending=False))

kbes=SelectKBest(score_func=chi2)
hasil1=kbes.fit(Xnya,datates['Best Performance'])
result1=pd.concat([pd.DataFrame(Xnya.columns),pd.DataFrame(hasil.scores_)],axis=1)
result1.columns=['Parameter','Score']
print('with chi squared')
print(result1.sort_values('Score',ascending=False))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
tree=ExtraTreesClassifier(n_estimators=200)
tree.fit(Xnya,datates['Best Performance'])
hasilnya=pd.concat([pd.DataFrame(Xnya.columns),pd.DataFrame(tree.feature_importances_)],axis=1)
hasilnya.columns=['Parameter','Scores']
print(hasilnya.sort_values('Scores',ascending=False))

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
XX=ss.fit_transform(Xnya)
tree=ExtraTreesClassifier(n_estimators=200)
tree.fit(XX,datates['Best Performance'])
hasilnya=pd.concat([pd.DataFrame(Xnya.columns),pd.DataFrame(tree.feature_importances_)],axis=1)
hasilnya.columns=['Parameter','Scores']
print(hasilnya.sort_values('Scores',ascending=False))

In [ ]:
import matplotlib.pyplot as plt
target=data.loc[:,'Best Performance']
key1=['Last_achievement_%','job_duration_from_training','age','gender','job_duration_in_current_branch','GPA','annual leave','number_of_dependences','branch_rotation']
key2=['job_duration_in_current_branch','job_duration_from_training','Achievement_above_100%_during3quartal','number_of_dependences','branch_rotation','job_rotation']
keycontinuous=key1
for item in keycontinuous:
    plt.boxplot(data[item])
    plt.title(item)
    plt.show()

In [ ]:
for item in data['Education_level'].unique():
    print(item,data['GPA'][data['Education_level']==item].median())

education level agak pengaruh (dri eda) jdi ambil 3 kelas besarnya dicoba one hot encoder, 

In [ ]:
datax=pd.concat([data.loc[:,keycontinuous],pd.get_dummies(data['Education_level']).drop(['level_0','level_2','level_5'],axis=1)],axis=1)
datax=data.loc[:,keycontinuous]
#datax.loc[:,'age']=datax['age'].apply(lambda x:2020-x)
print(datax)


In [ ]:
dataxtes=pd.concat([datatest.loc[:,keycontinuous],pd.get_dummies(datatest['Education_level']).drop(['level_0','level_2','level_5'],axis=1)],axis=1)
dataxtes=datatest.loc[:,keycontinuous]
#dataxtes.loc[:,'age']=dataxtes['age'].apply(lambda x:2020-x)
print(dataxtes)

In [ ]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
x1=ss.fit_transform(datax)
x1test=ss.fit_transform(dataxtes)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x1,target,stratify=target,random_state=123,test_size=0.3)
print((ytrain==1).sum())

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_precision_recall_curve
xg=xgb.XGBClassifier(n_estimators=100)
xg.fit(xtrain,ytrain)
print(classification_report(ytest,xg.predict(xtest)))
print(roc_auc_score(ytest,xg.predict_proba(xtest)[:,1]))
plot_precision_recall_curve(xg,xtest,ytest)
TPtes=pd.DataFrame(xtest[(ytest==1)&(xg.predict(xtest)==1),:],columns=datax.columns.tolist())
FPtes=pd.DataFrame(xtest[(ytest==0)&(xg.predict(xtest)==1),:],columns=datax.columns.tolist())
TNtes=pd.DataFrame(xtest[(ytest==0)&(xg.predict(xtest)==0),:],columns=datax.columns.tolist())
FNtes=pd.DataFrame(xtest[(ytest==1)&(xg.predict(xtest)==0),:],columns=datax.columns.tolist())

In [ ]:
from sklearn.metrics import classification_report

print((xg.predict(x1)==1).sum())
print(((xg.predict(x1)==1)&(target==1)).sum())
print('precision:',((xg.predict(x1)==1)&(target==1)).sum()/(xg.predict(x1)==1).sum())
print('recall:',((xg.predict(x1)==1)&(target==1)).sum()/(target==1).sum())
print((ytrain==1).sum())
print((target==1).sum())
print(data[xg.predict(x1)==1])
print(classification_report(target,xg.predict(x1)))


In [ ]:
import numpy as np
xg.fit(x1,target)
submit=pd.DataFrame({'index':np.arange(0,6000,1)})
submit['Best Performance']=xg.predict_proba(x1test)[:,1]
submit.set_index('index',inplace=True)
print(submit)
submit.to_csv('resul.csv')
print((xg.predict(x1test)==1).sum())
print((submit['Best Performance']>0.5).sum())

In [ ]:
print(len(Xnya.columns.tolist()),',',len(xg.feature_importances_))
importance=pd.DataFrame({'Columns':datax.columns.tolist(),'Importance':xg.feature_importances_})
print(importance)


In [ ]:
import seaborn as sns
for item in datax.columns.tolist():
    print(item)
    sns.distplot(TPtes[item],hist=False,color='g')
    sns.distplot(TNtes[item],hist=False,color='b')
    sns.distplot(FPtes[item],hist=False,color='y')
    sns.distplot(FNtes[item],hist=False,color='r')
    plt.title(item)
    plt.show()